In [8]:
# Import necessary libraries for data processing, database connection, and anonymization
import pandas as pd  # For data manipulation
import hashlib      # For secure hashing of IDs
import random       # For generating random values
import os           # For accessing environment variables
from datetime import datetime  # For timestamp operations
from faker import Faker  # For generating fake names/IDs
import sqlalchemy   # For database operations
from sqlalchemy import create_engine, exc  # For database connection
from dotenv import load_dotenv  # For loading environment variables
import ipywidgets as widgets  # For interactive UI elements
from IPython.display import display, clear_output  # For displaying output in notebook

# Load environment variables
load_dotenv()

# Cell 2: Enhanced database connection setup
# Function to create a secure database connection with multiple fallback options
def create_db_connection():
    """
    Create a secure database connection using environment variables
    with improved error handling and fallback options
    """
    # Configuration with fallback values if environment variables aren't set
    config = {
        'host': os.getenv('DB_HOST', '(localdb)/MSSQLLocalDB'),  # Database server address
        'name': os.getenv('DB_NAME', 'BigProject'),  # Database name
        'user': os.getenv('DB_USER', 'Bonga Sikhakhane'),  # Username
        'password': os.getenv('DB_PASS', ''),  # Password
        'port': os.getenv('DB_PORT', '1433'),  # Port number
        'driver': os.getenv('DB_DRIVER', 'ODBC Driver 17 for SQL Server')  # ODBC driver
    }
    
    # Two different connection string formats to try:
    connection_strings = [
        # 1. Windows Authentication (uses current Windows credentials)
        f"mssql+pyodbc://{config['host']}/{config['name']}?"
        f"driver={config['driver']}&trusted_connection=yes",
        
        # 2. SQL Authentication (uses username/password)
        f"mssql+pyodbc://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['name']}?"
        f"driver={config['driver']}"
    ]

    # Try each connection method until one works
    for conn_str in connection_strings:
        try:
            engine = create_engine(conn_str, pool_pre_ping=True)
            # Test the connection with a simple query
            with engine.connect() as test_conn:
                test_conn.execute("SELECT 1")
            print(f"✅ Connected using: {conn_str.split('?')[0]}...")
            return engine
        except exc.SQLAlchemyError as e:
            print(f"⚠️ Attempt failed: {str(e)}")
            continue
    
    # If all connection attempts fail
    raise ConnectionError("Could not establish database connection with any method")

# Test the connection with enhanced error messages
try:
    print("🔄 Attempting database connection...")
    engine = create_db_connection()
    
    # Verify required tables exist in the database
    inspector = sqlalchemy.inspect(engine)
    required_tables = ['Researcher', 'Participant', 'Institution']
    missing_tables = [t for t in required_tables if t not in inspector.get_table_names()]
    
    if missing_tables:
        print(f"⚠️ Warning: Missing tables - {', '.join(missing_tables)}")
    else:
        print("✅ Database structure validation passed!")
        
except Exception as e:
    print(f"❌ Critical error: {str(e)}")
    print("\nTroubleshooting steps:")
    print("1. Verify SQL Server is running")
    print("2. Check firewall settings")
    print("3. Validate credentials in .env file")
    print("4. Ensure ODBC driver is installed") 

🔄 Attempting database connection...
⚠️ Attempt failed: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The parameter is incorrect.\r\n (87) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (87)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)
⚠️ Attempt failed: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The parameter is incorrect.\r\n (87) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Micros

In [9]:
# Class that handles all participant anonymization tasks
class ParticipantAnonymizer:
    def __init__(self, engine):
        self.engine = engine  # Database connection
        self.faker = Faker()  # For generating fake data
        self.mapping_df = None  # Will store mapping between real and fake IDs
        
    def generate_mapping(self, participant_ids):
        """Generate secure mapping between original and anonymized IDs"""
        data = {
            'original_id': [],  # Store original participant IDs
            'pseudonym': [],    # Generate readable fake IDs (P-AB-12-345)
            'hashed_id': [],   # Store cryptographically hashed IDs
            'random_id': []    # Store completely random numeric IDs
        }
        
        for pid in participant_ids:
            # Get salt from environment variables for extra security
            salt = os.getenv('ANONYMIZATION_SALT', 'default-secure-salt')
            
            # Store original ID
            data['original_id'].append(pid)
            # Generate fake ID with pattern P-XX-99-999
            data['pseudonym'].append(f"P-{self.faker.unique.bothify(text='??-##-###').upper()}")
            # Create secure hash of original ID
            data['hashed_id'].append(
                hashlib.pbkdf2_hmac('sha256', str(pid).encode(), salt.encode(), 100000).hex()
            )
            # Generate random number between 100000-999999
            data['random_id'].append(random.randint(100000, 999999))
            
        # Store mapping as a DataFrame
        self.mapping_df = pd.DataFrame(data)
        return self.mapping_df
    
    def anonymize_participants(self):
        """Main anonymization workflow"""
        try:
            # Load all participant data from database
            participants = pd.read_sql("SELECT * FROM Participant", self.engine)
            
            if participants.empty:
                print("⚠️ No participant data found")
                return None
                
            print(f"🔍 Found {len(participants)} participants to anonymize")
            
            # Generate mapping between real and fake IDs
            mapping = self.generate_mapping(participants['ParticipantID'])
            
            # Merge original data with mapping to create anonymized version
            anonymized = participants.merge(
                mapping, 
                left_on='ParticipantID', 
                right_on='original_id'
            ).drop(columns=['Name', 'original_id'])  # Remove sensitive name column
            
            # Add slight random noise to age values for extra privacy
            if 'Age' in anonymized.columns:
                anonymized['Age'] = anonymized['Age'].apply(
                    lambda x: x + random.randint(-2, 2) if not pd.isna(x) else x
                )
            
            print("✅ Anonymization complete")
            return anonymized
            
        except Exception as e:
            print(f"❌ Error during anonymization: {str(e)}")
            return None

In [10]:
# Creates an interactive button in the Jupyter notebook
def run_anonymization():
    """Interactive widget for running anonymization"""
    # Create a button widget
    button = widgets.Button(description="Anonymize Data")
    # Create an output area for messages
    output = widgets.Output()
    
    # Define what happens when button is clicked
    def on_button_click(b):
        with output:
            clear_output()
            print("⏳ Starting anonymization process...")
            
            try:
                # Create anonymizer instance and run it
                anonymizer = ParticipantAnonymizer(engine)
                result = anonymizer.anonymize_participants()
                
                if result is not None:
                    # Show sample of anonymized data
                    display(result.head(3))
                    
                    # Add a second button to save results
                    save_button = widgets.Button(description="Save to Database")
                    
                    # Define save button action
                    def on_save_click(b):
                        with output:
                            try:
                                # Save anonymized data to new table
                                result.to_sql(
                                    'AnonymizedParticipants',
                                    engine,
                                    if_exists='replace',  # Overwrite if exists
                                    index=False
                                )
                                print("💾 Successfully saved anonymized data to database")
                            except Exception as e:
                                print(f"❌ Failed to save: {str(e)}")
                    
                    save_button.on_click(on_save_click)
                    display(save_button)
                    
            except Exception as e:
                print(f"❌ Critical error: {str(e)}")
    
    # Connect button click to function
    button.on_click(on_button_click)
    # Display the button and output area
    display(button, output)

# Run the interactive UI
run_anonymization()

Button(description='Anonymize Data', style=ButtonStyle())

Output()